In [15]:
import pandas as pd
import numpy as np
import math
import statistics as st
from datetime import timedelta

In [16]:
df = pd.read_excel("../datasets/indian_stock_indices/Stock_Indices.xlsx")

In [17]:
df.head()

,Date,SENSEX,NIFTY50,NIFTY_CONSUMPTION
0,2014-01-01,21140.48,6323.80,2590.000000
1,2014-01-02,20888.33,6301.25,2544.649902
2,2014-01-03,20851.33,6194.55,2546.949951
3,2014-01-06,20787.30,6220.85,2544.649902
4,2014-01-07,20693.24,6203.90,2543.399902


In [18]:
shape = (64, 64) # We can have 64 separate patterns 

pattern_graph = np.zeros(shape)

In [19]:
ONE_DAY = timedelta(days=1)

### Calculating weekly returns and volatilities (over 1 week) for SENSEX, NIFTY50 and NIFTY Consumption

In [20]:
N = 2469

sensex_volatilities = []
nifty_volatilities = []
niftyc_volatilities = []

sensex_returns = []
nifty_returns = []
niftyc_returns = []

i = 0
j = 0

while i < N:
    sensex_weekly_returns = []
    nifty_weekly_returns = []
    niftyc_weekly_returns = []
    
    week_size = 1
    
    while ((j + 1) < N) and ((df['Date'][j + 1] - df['Date'][j]) == timedelta(days=1)):
        week_size += 1
#         print(df['SENSEX'][j])

        sensex_ret = math.log(df['SENSEX'][j + 1] / df['SENSEX'][j])
        nifty_ret = math.log(df['NIFTY50'][j + 1] / df['NIFTY50'][j])
        niftyc_ret = math.log(df['NIFTY_CONSUMPTION'][j + 1] / df['NIFTY_CONSUMPTION'][j])
        
        sensex_weekly_returns.append(sensex_ret)
        nifty_weekly_returns.append(nifty_ret)
        niftyc_weekly_returns.append(niftyc_ret)
        
        j += 1
    j += 1
    if len(sensex_weekly_returns) > 1: # One can use try-catch block here
        sensex_vol = st.stdev(sensex_weekly_returns) * math.sqrt(week_size)
        nifty_vol = st.stdev(nifty_weekly_returns) * math.sqrt(week_size)
        niftyc_vol = st.stdev(niftyc_weekly_returns) * math.sqrt(week_size)
        
    sensex_volatilities.append(sensex_vol)
    nifty_volatilities.append(nifty_vol)
    niftyc_volatilities.append(niftyc_vol)
    
#     print("\n")
    if j < N: # One can use try-catch block here
        sensex_weekly_return = math.log(df['SENSEX'][j] / df['SENSEX'][j - week_size])
        nifty_weekly_return = math.log(df['NIFTY50'][j] / df['NIFTY50'][j - week_size])
        niftyc_weekly_return = math.log(df['NIFTY_CONSUMPTION'][j] / df['NIFTY_CONSUMPTION'][j - week_size])
    sensex_returns.append(sensex_weekly_return)
    nifty_returns.append(nifty_weekly_return)
    niftyc_returns.append(nifty_weekly_return)
    i = j

Testing cell

In [24]:
# n = len(sensex_volatilities)
# m = len(sensex_returns)

# for i in range(n):
#     print(sensex_volatilities[i], nifty_volatilities[i], niftyc_volatilities[i])

# print("\n")
# for i in range(m):
#     print(sensex_returns[i], nifty_returns[i], niftyc_returns[i])

In [28]:
sensex_avg_volatility = np.average(sensex_volatilities)
nifty_avg_volatility = np.average(nifty_volatilities)
niftyc_avg_volatility = np.average(niftyc_volatilities)

In [37]:
N = len(sensex_returns)

prev_sensex_base_value = 0
prev_nifty_base_value = 0
prev_niftyc_base_value = 0

if sensex_returns[0] >= 0 and sensex_volatilities[0] >= sensex_avg_volatility:
    prev_sensex_base_value = 0
elif sensex_returns[0] >= 0:
    prev_sensex_base_value = 1
elif sensex_returns[0] < 0 and sensex_volatilities[0] >= sensex_avg_volatility:
    prev_sensex_base_value = 2
else:
    prev_sensex_base_value = 3
    
if nifty_returns[0] >= 0 and nifty_volatilities[0] >= nifty_avg_volatility:
    prev_nifty_base_value = 0
elif nifty_returns[0] >= 0:
    prev_nifty_base_value = 1
elif nifty_returns[0] < 0 and nifty_volatilities[0] >= nifty_avg_volatility:
    prev_nifty_base_value = 2
else:
    prev_nifty_base_value = 3

if niftyc_returns[0] >= 0 and niftyc_volatilities[0] >= niftyc_avg_volatility:
    prev_niftyc_base_value = 0
elif niftyc_returns[0] >= 0:
    prev_niftyc_base_value = 1
elif niftyc_returns[0] < 0 and niftyc_volatilities[0] >= niftyc_avg_volatility:
    prev_niftyc_base_value = 2
else:
    prev_niftyc_base_value = 3
    
    
for i in range(1, N):
    
    next_sensex_base_value = 0
    next_nifty_base_value = 0
    next_niftyc_base_value = 0

    if sensex_returns[i] >= 0 and sensex_volatilities[i] >= sensex_avg_volatility:
        next_sensex_base_value = 0
    elif sensex_returns[i] >= 0:
        next_sensex_base_value = 1
    elif sensex_returns[i] < 0 and sensex_volatilities[i] >= sensex_avg_volatility:
        next_sensex_base_value = 2
    else:
        next_sensex_base_value = 3

    if nifty_returns[i] >= 0 and nifty_volatilities[i] >= nifty_avg_volatility:
        next_nifty_base_value = 0
    elif nifty_returns[i] >= 0:
        next_nifty_base_value = 1
    elif nifty_returns[i] < 0 and nifty_volatilities[i] >= nifty_avg_volatility:
        next_nifty_base_value = 2
    else:
        next_nifty_base_value = 3

    if niftyc_returns[i] >= 0 and niftyc_volatilities[i] >= niftyc_avg_volatility:
        next_niftyc_base_value = 0
    elif niftyc_returns[i] >= 0:
        next_niftyc_base_value = 1
    elif niftyc_returns[i] < 0 and niftyc_volatilities[i] >= niftyc_avg_volatility:
        next_niftyc_base_value = 2
    else:
        next_niftyc_base_value = 3
    
    prev_index = prev_niftyc_base_value * 4 ** 0 + prev_nifty_base_value * 4 ** 1 + prev_sensex_base_value * 4 ** 2
    next_index = next_niftyc_base_value * 4 ** 0 + next_nifty_base_value * 4 ** 1 + next_sensex_base_value * 4 ** 2
    
    pattern_graph[prev_index][next_index] += 1
    pattern_graph[next_index][prev_index] += 1
    
    prev_niftyc_base_value = next_niftyc_base_value
    prev_nifty_base_value = next_nifty_base_value
    prev_sensex_base_value = next_sensex_base_value
    

In [38]:
print(pd.DataFrame(pattern_graph))

      0    1    2    3     4    5    6    7    8    9   ...   54   55   56  \
0   56.0  2.0  0.0  0.0   2.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1    2.0  8.0  0.0  0.0   1.0  2.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2    0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4    2.0  1.0  0.0  0.0  49.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
..   ...  ...  ...  ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
59   0.0  1.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
60   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
61   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
62   5.0  2.0  0.0  0.0   2.0  2.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
63  10.0  3.0  0.0  0.0   1.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     57   58   59   60   61    62     63  
0   0.0  1.0  0.0  0